In [1]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh


import numpy as np

In [14]:
class camera:
    def __init__(self, index, mode = "dev"):
        self.cap = cv2.VideoCapture(index)
        self.face_mesh = mp_face_mesh.FaceMesh(
            max_num_faces=1,
            refine_landmarks=True,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5)     
        self.success, self.image = self.cap.read()
        self.results = self.face_mesh.process(self.image)
        self.loop = 0
        self.mode = mode
    def camera_update(self):
        self.success, self.image = self.cap.read()
    
    def get_face_mesh_data(self):
        self.image.flags.writeable = False
        self.image = cv2.cvtColor(self.image, cv2.COLOR_BGR2RGB)
        self.results = self.face_mesh.process(self.image)
    
    def draw_face_mesh_data(self):
        self.image.flags.writeable = True
        self.image = cv2.cvtColor(self.image, cv2.COLOR_RGB2BGR)
        if self.results.multi_face_landmarks:
            for face_landmarks in self.results.multi_face_landmarks:
                mp_drawing.draw_landmarks(
                    image=self.image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_tesselation_style())
                mp_drawing.draw_landmarks(
                    image=self.image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_contours_style())
                mp_drawing.draw_landmarks(
                    image=self.image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_IRISES,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_iris_connections_style())
                # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Hands', cv2.flip(self.image, 1))
    
    def get_data(self):
        for face_landmarks in self.results.multi_face_landmarks:
            result = face_landmarks.landmark
        return result
    
    def release(self):
        self.cap.release()
    
    def calculate_face_direciton(self,data):
        # 143 : right eye end , 272 : left eye end , 199 : jaw end
        if(self.mode == "dev"):
            print("143 : ",[data[143].x,data[143].y,data[143].z]," 272: ",[data[272].x,data[272].y,data[272].z]," 199 : ",[data[199].x,data[199].y,data[199].z])
        vector_a = np.array([data[199].x,data[199].y,data[199].z]) - np.array([data[143].x,data[143].y,data[143].z])                          
        vector_b = np.array([data[272].x,data[272].y,data[272].z]) - np.array([data[199].x,data[199].y,data[199].z])
        if(self.mode == "dev"):
            print("vector a : ",vector_a," vector b : ",vector_b)
        result = np.cross(vector_a,vector_b) 
        if(self.mode == "dev"):
            print("origin result vector : ",result)
        return result / np.linalg.norm(result)
        
        
    def run(self):
        while self.cap.isOpened():
            self.camera_update()
            if not self.success:
                print("Ignoring empty camera frame.")
                # If loading a video, use 'break' instead of 'continue'.
                continue
            
            self.get_face_mesh_data()
            
            self.draw_face_mesh_data()
            
            if(self.results.multi_face_landmarks != None):
                print(self.loop)
                data = self.get_data()
                face_direction = self.calculate_face_direciton(data)
                print("direction :\n",face_direction)
                
            self.loop+=1
            if cv2.waitKey(5) & 0xFF == 27:
                break
        
        self.release()

In [16]:
camera1 = camera(0, "build")
camera1.run()

14
direction :
 [-0.22947517 -0.56440191 -0.79296383]
15
direction :
 [-0.46604881 -0.5332562  -0.70600023]
16
direction :
 [-0.46791716 -0.52240783 -0.71284191]
17
direction :
 [-0.45680226 -0.4985745  -0.73671919]
18
direction :
 [-0.45222021 -0.51167187 -0.73054006]
19
direction :
 [-0.43140409 -0.51406921 -0.74136587]
20
direction :
 [-0.42720721 -0.4948307  -0.75672754]
21
direction :
 [-0.40046437 -0.48132176 -0.77971639]
22
direction :
 [-0.39037087 -0.47128009 -0.79088916]
23
direction :
 [-0.38387007 -0.45667888 -0.80255104]
24
direction :
 [-0.38338211 -0.44769355 -0.80782959]
25
direction :
 [-0.38510167 -0.44234562 -0.80995497]
26
direction :
 [-0.37835958 -0.41688942 -0.82646672]
27
direction :
 [-0.36646543 -0.42658329 -0.82687954]
28
direction :
 [-0.36257918 -0.41141434 -0.83622639]
29
direction :
 [-0.38696426 -0.40135969 -0.83016207]
30
direction :
 [-0.35755229 -0.42180374 -0.83320944]
31
direction :
 [-0.34386423 -0.42208065 -0.83881185]
32
direction :
 [-0.33348979

166
direction :
 [-0.66905924 -0.31315338 -0.67401387]
167
direction :
 [-0.60984902 -0.48734012 -0.62496702]


KeyboardInterrupt: 